# **Import Library**

In [ ]:
pip install --upgrade git+https://github.com/ariaghora/mpstemmer.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ariaghora/mpstemmer.git to /tmp/pip-req-build-274x9wyd
  Running command git clone -q https://github.com/ariaghora/mpstemmer.git /tmp/pip-req-build-274x9wyd


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import string

In [ ]:
import warnings
warnings.filterwarnings('ignore')

Pada bagian ini, kita akan meng-import beberapa library dasar yang sering digunakan pada pengolahan data. Library-library tersebut, yaitu Pandas dan Numpy untuk pengolahan dan manipulasi data baik dalam bentuk Array maupun DataFrame, Matplotlib dan Seaborn untuk membuat visualisasi data, dan re beserta string untuk melakukan RegEx dan pengolahan teks.

# **Import Dataset**

In [ ]:
!gdown --id 1ZSvxGmMJfN0Yhnx5TW2vP2-t5Vt1oY7w

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ZSvxGmMJfN0Yhnx5TW2vP2-t5Vt1oY7w
To: /content/Kel5_Unlabelled.xlsx
100% 617k/617k [00:00<00:00, 50.3MB/s]


In [ ]:
pd.set_option('display.max_rows', None)

df = pd.read_excel('/content/Kel5_Unlabelled.xlsx')
df.head()

,Judul,Isi,Label_Hoax,Label_Adam,Label_gee,Label_Ratu,Label_Kiki,Label_Anis,Total_Nilai,Label_Majority,Label_Akhir
0,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,1.0,1.0,0.0,1.0,1.0,4.0,1.0,999
4,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,1.0,1.0,0.0,1.0,1.0,1.0,4.0,1.0,999


# **Data Preprocessing**

## Rename & Subset Columns

In [ ]:
df_hoax = df[['Isi', 'Label_Hoax']]

## ==== PENTING ====
df_hoax = df_hoax.head(4231)

df_hoax.head()

,Isi,Label_Hoax
0,A caller to a radio talk show recently shared ...,1.0
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0
4,Kadrun kalo lihat foto ini panas dingin . .,1.0


# **Text Preprocessing**

Pada bagian ini, akan dilakukan preprocessing terlebih dahulu agar data teks yang nantinya akan dianalisis sudah bersih, memiliki informasi yang relevan, menghapus teks yang tidak relevan, dan menghasilkan suatu insight yang menarik. Tahapan-tahapan preprocessing tersebut, yaitu:

## Case Folding

Pada bagian ini, seluruh teks akan dibuat menjadi lowercase agar menjadi sama dan konsisten. Selain itu, proses ini dilakukan karena seperti yang kita tahu sendiri di Python ada sifat case-sensitive di mana kata “Kucing” akan berbeda dengan kata “kucing”. Oleh karena itu, proses case folding dilakukan untuk mengatasi hal tersebut

In [ ]:
df_hoax['Isi_Casefold'] = df_hoax['Isi'].str.lower()
df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k..."
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .


Berdasarkan output di atas, dapat dilihat bahwa semua huruf pada teks sudah menjadi huruf kecil semua.

## Text Cleaning

Pada bagian ini, seluruh informasi yang tidak relevan dalam teks akan dilakukan penghapusan dengan menggunakan RegEx. Informasi-informasi tidak relevan yang dihapus tersebut meliputi, yaitu angka, tanda baca, spasi kosong, dan kata dengan huruf tunggal.

In [ ]:
# remove emoji
df_hoax['Isi_Relevant'] = df_hoax['Isi_Casefold'].astype(str).apply(lambda x: x.encode('latin-1', 'ignore').decode('latin-1'))

def remove_mention(text):
  return re.sub(r'@[A-Za-z0-9]+\s?', '', str(text))

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_mention)

def remove_hashtag(text):
  return re.sub(r'#[A-Za-z0-9]+\s?', '', str(text))

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_hashtag)

def remove_https(text):
  return re.sub(r'https:\/\/.*', '', str(text))

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_https)

def remove_number(text):
  return re.sub(r'\d+', '', str(text))

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_number)

def remove_punc(text):
  return text.translate(str.maketrans('','',string.punctuation+"“"+"🫶"))

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_punc)

def remove_whitespace(text):
  return text.strip()

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_whitespace)

def remove_whitespace_multi(text):
  return re.sub('\s+', ' ', text)

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_whitespace_multi)

def remove_single_char(text):
  return re.sub(r'\b[a-zA-Z]\b', '', text)

df_hoax['Isi_Relevant'] = df_hoax['Isi_Relevant'].apply(remove_single_char)

df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold,Isi_Relevant
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...,caller to radio talk show recently shared th...
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...,ythseluruh anggota grup sesuai instruksi guber...
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...,jokowi adalah presiden terbaik dlm sejarah ban...
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",maaf mas dan mbak ini bukan politik tapi kenya...
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .,kadrun kalo lihat foto ini panas dingin


Berdasarkan output di atas, dapat dilihat bahwa data teks sudah cukup bersih di mana informasi-informasi tidak relevan seperti angka, tanda baca, spasi kosong, dan kata dengan huruf tunggal sudah berhasil dihapus.

## Tokenizing

Pada bagian ini, akan dilakukan proses pemisahan teks menjadi bagian yang lebih kecil yang dalam hal ini teks akan dipisah menjadi kata per kata. 

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import word_tokenize

def word_tokenize_wrapper(text):
  return word_tokenize(text)

df_hoax['Isi_Tokenized'] = df_hoax['Isi_Relevant'].apply(word_tokenize_wrapper)

df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold,Isi_Relevant,Isi_Tokenized
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...,caller to radio talk show recently shared th...,"[caller, to, radio, talk, show, recently, shar..."
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...,ythseluruh anggota grup sesuai instruksi guber...,"[ythseluruh, anggota, grup, sesuai, instruksi,..."
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...,jokowi adalah presiden terbaik dlm sejarah ban...,"[jokowi, adalah, presiden, terbaik, dlm, sejar..."
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",maaf mas dan mbak ini bukan politik tapi kenya...,"[maaf, mas, dan, mbak, ini, bukan, politik, ta..."
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .,kadrun kalo lihat foto ini panas dingin,"[kadrun, kalo, lihat, foto, ini, panas, dingin]"


Berdasarkan output di atas, dapat dilihat bahwa proses pemisahan menjadi kata atau tokenizing sudah berhasil dilakukan.

## Stemming

Pada bagian ini, akan dilakukan proses stemming di mana suatu kata akan diubah menjadi bentuk dasarnya, misal kata “memakan”, “dimakan”, dan “termakan” akan diubah menjadi kata “makan”. 

In [ ]:
!pip install Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from mpstemmer import MPStemmer

stemmer = MPStemmer()

def stemming(words):
  return [stemmer.stem(word) for word in words]

df_hoax['Isi_Stemmed'] = df_hoax['Isi_Tokenized'].apply(stemming)
df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold,Isi_Relevant,Isi_Tokenized,Isi_Stemmed
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...,caller to radio talk show recently shared th...,"[caller, to, radio, talk, show, recently, shar...","[caller, to, radio, talk, show, recently, shar..."
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...,ythseluruh anggota grup sesuai instruksi guber...,"[ythseluruh, anggota, grup, sesuai, instruksi,...","[ythseluruh, anggota, grup, sesuai, instruksi,..."
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...,jokowi adalah presiden terbaik dlm sejarah ban...,"[jokowi, adalah, presiden, terbaik, dlm, sejar...","[jokowi, adalah, presiden, baik, dlm, sejarah,..."
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",maaf mas dan mbak ini bukan politik tapi kenya...,"[maaf, mas, dan, mbak, ini, bukan, politik, ta...","[maaf, mas, dan, mbak, ini, bukan, politik, ta..."
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .,kadrun kalo lihat foto ini panas dingin,"[kadrun, kalo, lihat, foto, ini, panas, dingin]","[kadrun, kalau, lihat, foto, ini, panas, dingin]"


Berdasarkan output di atas, dapat dilihat bahwa kata “pengawas” sudah menjadi “awas”,dan “kesehatan” sudah menjadi “sehat”. Dari hal tersebut, kita sudah dapat memastikan bahwa masing-masing kata sudah dalam bentuk dasarnya.

## Stopwords Removal

Pada bagian ini, akan dilakukan proses filtering di mana kita akan menghapus kata-kata yang sering keluar tetapi tidak relevan. Kata-kata tersebut seperti “yang”, “adalah”, “yaitu”, dan lain sebagainya

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
list_stopwords = stopwords.words('indonesian')
list_stopwords = set(list_stopwords)
print(list_stopwords)

{'antara', 'cukup', 'tuturnya', 'yakni', 'toh', 'tertentu', 'melihat', 'umum', 'sekaligus', 'memperlihatkan', 'sepertinya', 'setinggi', 'begitulah', 'menyampaikan', 'saya', 'sebabnya', 'sedangkan', 'juga', 'mengetahui', 'bertanya', 'soalnya', 'menunjukkan', 'wah', 'serupa', 'cuma', 'supaya', 'lagian', 'mendapatkan', 'nyaris', 'semacam', 'adalah', 'bersiap-siap', 'macam', 'tidak', 'sesudah', 'masalahnya', 'kedua', 'mengapa', 'umumnya', 'setiap', 'dituturkan', 'untuk', 'enggak', 'diibaratkan', 'diungkapkan', 'dahulu', 'dini', 'bagian', 'kepadanya', 'sejauh', 'semaunya', 'ucap', 'ujarnya', 'tempat', 'sama-sama', 'diperkirakan', 'entahlah', 'dimaksudnya', 'ingin', 'diberi', 'mengibaratkannya', 'lah', 'bertutur', 'terutama', 'janganlah', 'se', 'aku', 'tetapi', 'mendatangkan', 'semampunya', 'menggunakan', 'bulan', 'sebagaimana', 'menghendaki', 'haruslah', 'pertama-tama', 'didatangkan', 'sudah', 'sekurang-kurangnya', 'panjang', 'dituturkannya', 'berakhir', 'menuju', 'sudahlah', 'beginian', 'p

In [ ]:
def remove_stopwords(words):
  return [word for word in words if word not in list_stopwords]

df_hoax['Isi_Clean'] = df_hoax['Isi_Stemmed'].apply(remove_stopwords)

df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold,Isi_Relevant,Isi_Tokenized,Isi_Stemmed,Isi_Clean
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...,caller to radio talk show recently shared th...,"[caller, to, radio, talk, show, recently, shar...","[caller, to, radio, talk, show, recently, shar...","[caller, to, radio, talk, show, recently, shar..."
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...,ythseluruh anggota grup sesuai instruksi guber...,"[ythseluruh, anggota, grup, sesuai, instruksi,...","[ythseluruh, anggota, grup, sesuai, instruksi,...","[ythseluruh, anggota, grup, sesuai, instruksi,..."
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...,jokowi adalah presiden terbaik dlm sejarah ban...,"[jokowi, adalah, presiden, terbaik, dlm, sejar...","[jokowi, adalah, presiden, baik, dlm, sejarah,...","[jokowi, presiden, dlm, sejarah, bangsa, indon..."
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",maaf mas dan mbak ini bukan politik tapi kenya...,"[maaf, mas, dan, mbak, ini, bukan, politik, ta...","[maaf, mas, dan, mbak, ini, bukan, politik, ta...","[maaf, mas, mbak, politik, nyata, jokow, hasil..."
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .,kadrun kalo lihat foto ini panas dingin,"[kadrun, kalo, lihat, foto, ini, panas, dingin]","[kadrun, kalau, lihat, foto, ini, panas, dingin]","[kadrun, lihat, foto, panas, dingin]"


Berdasarkan output di atas, dapat dilihat bahwa kata-kata yang termasuk ke dalam stopwords seperti kata “pada” sudah dihapus dari data

In [ ]:
df_hoax['Isi_Clean_Unlisted'] = df_hoax['Isi_Clean'].agg(lambda x: ' '.join(map(str, x)))
df_hoax.head()

,Isi,Label_Hoax,Isi_Casefold,Isi_Relevant,Isi_Tokenized,Isi_Stemmed,Isi_Clean,Isi_Clean_Unlisted
0,A caller to a radio talk show recently shared ...,1.0,a caller to a radio talk show recently shared ...,caller to radio talk show recently shared th...,"[caller, to, radio, talk, show, recently, shar...","[caller, to, radio, talk, show, recently, shar...","[caller, to, radio, talk, show, recently, shar...",caller to radio talk show recently shared that...
1,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,1.0,yth.seluruh anggota grup sesuai instruksi gube...,ythseluruh anggota grup sesuai instruksi guber...,"[ythseluruh, anggota, grup, sesuai, instruksi,...","[ythseluruh, anggota, grup, sesuai, instruksi,...","[ythseluruh, anggota, grup, sesuai, instruksi,...",ythseluruh anggota grup sesuai instruksi guber...
2,Jokowi adalah presiden terbaik dlm sejarah ban...,1.0,jokowi adalah presiden terbaik dlm sejarah ban...,jokowi adalah presiden terbaik dlm sejarah ban...,"[jokowi, adalah, presiden, terbaik, dlm, sejar...","[jokowi, adalah, presiden, baik, dlm, sejarah,...","[jokowi, presiden, dlm, sejarah, bangsa, indon...",jokowi presiden dlm sejarah bangsa indonesia j...
3,"Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",1.0,"maaf mas2 dan mbak2, ini bukan politik, tapi k...",maaf mas dan mbak ini bukan politik tapi kenya...,"[maaf, mas, dan, mbak, ini, bukan, politik, ta...","[maaf, mas, dan, mbak, ini, bukan, politik, ta...","[maaf, mas, mbak, politik, nyata, jokow, hasil...",maaf mas mbak politik nyata jokow hasil pulang...
4,Kadrun kalo lihat foto ini panas dingin . .,1.0,kadrun kalo lihat foto ini panas dingin . .,kadrun kalo lihat foto ini panas dingin,"[kadrun, kalo, lihat, foto, ini, panas, dingin]","[kadrun, kalau, lihat, foto, ini, panas, dingin]","[kadrun, lihat, foto, panas, dingin]",kadrun lihat foto panas dingin


# **Modelling Preparation**

## TF-IDF & N-Grams

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
cv_unigrams = TfidfVectorizer(ngram_range = (1,1), max_features=5000)
cv_bigrams = TfidfVectorizer(ngram_range = (2,2), max_features=5000)
cv_trigrams = TfidfVectorizer(ngram_range = (3,3), max_features=5000)

X_unigram = cv_unigrams.fit_transform(df_hoax['Isi_Clean_Unlisted']).toarray()
y_unigram = df_hoax['Label_Hoax'].values

X_bigram = cv_bigrams.fit_transform(df_hoax['Isi_Clean_Unlisted']).toarray()
y_bigram = df_hoax['Label_Hoax'].values

X_trigram = cv_trigrams.fit_transform(df_hoax['Isi_Clean_Unlisted']).toarray()
y_trigram = df_hoax['Label_Hoax'].values

## Imbalance

In [ ]:
df_hoax['Label_Hoax'].value_counts()

1.0    3465
0.0     766
Name: Label_Hoax, dtype: int64

In [ ]:
import imblearn

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 42)
X_smote_unigram, y_smote_unigram = sm.fit_resample(X_unigram, y_unigram)
X_smote_bigram, y_smote_bigram = sm.fit_resample(X_bigram, y_bigram)
X_smote_trigram, y_smote_trigram = sm.fit_resample(X_trigram, y_trigram)

## Split Dataset

In [ ]:
from sklearn.model_selection import train_test_split
  
# split into 70:30 ration
X_train_uni, X_test_uni, y_train_uni, y_test_uni = train_test_split(X_smote_unigram, 
                                                                    y_smote_unigram, 
                                                                    test_size = 0.2, 
                                                                    random_state = 42)
X_train_bi, X_test_bi, y_train_bi, y_test_bi = train_test_split(X_smote_bigram, 
                                                                y_smote_bigram, 
                                                                test_size = 0.2, 
                                                                random_state = 42)
X_train_tri, X_test_tri, y_train_tri, y_test_tri = train_test_split(X_smote_trigram, 
                                                                    y_smote_trigram, 
                                                                    test_size = 0.2, 
                                                                    random_state = 42)

# **Modelling**

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier_uni = MultinomialNB()
nb_classifier_bi = MultinomialNB()
nb_classifier_tri = MultinomialNB()
nb_classifier_uni.fit(X_train_uni, y_train_uni.ravel())
nb_classifier_bi.fit(X_train_bi, y_train_bi.ravel())
nb_classifier_tri.fit(X_train_tri, y_train_tri.ravel())

y_pred_nb_uni = nb_classifier_uni.predict(X_test_uni)
y_pred_nb_bi = nb_classifier_bi.predict(X_test_bi)
y_pred_nb_tri = nb_classifier_tri.predict(X_test_tri)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_nb_uni = round(accuracy_score(y_test_uni, y_pred_nb_uni),2)
precision_nb_uni = round(precision_score(y_test_uni, y_pred_nb_uni, average='weighted'),2)
recall_nb_uni = round(recall_score(y_test_uni, y_pred_nb_uni, average='weighted'),2)
f1score_nb_uni = round(f1_score(y_test_uni, y_pred_nb_uni, average='weighted'),2)

akurasi_nb_bi = round(accuracy_score(y_test_bi, y_pred_nb_bi),2)
precision_nb_bi = round(precision_score(y_test_bi, y_pred_nb_bi, average='weighted'),2)
recall_nb_bi = round(recall_score(y_test_bi, y_pred_nb_bi, average='weighted'),2)
f1score_nb_bi = round(f1_score(y_test_bi, y_pred_nb_bi, average='weighted'),2)

akurasi_nb_tri = round(accuracy_score(y_test_tri, y_pred_nb_tri),2)
precision_nb_tri = round(precision_score(y_test_tri, y_pred_nb_tri, average='weighted'),2)
recall_nb_tri = round(recall_score(y_test_tri, y_pred_nb_tri, average='weighted'),2)
f1score_nb_tri = round(f1_score(y_test_tri, y_pred_nb_tri, average='weighted'),2)

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier_uni = RandomForestClassifier(random_state = 42)
rf_classifier_bi = RandomForestClassifier(random_state = 42)
rf_classifier_tri = RandomForestClassifier(random_state = 42)
rf_classifier_uni.fit(X_train_uni, y_train_uni.ravel())
rf_classifier_bi.fit(X_train_bi, y_train_bi.ravel())
rf_classifier_tri.fit(X_train_tri, y_train_tri.ravel())

y_pred_rf_uni = rf_classifier_uni.predict(X_test_uni)
y_pred_rf_bi = rf_classifier_bi.predict(X_test_bi)
y_pred_rf_tri = rf_classifier_tri.predict(X_test_tri)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_rf_uni = round(accuracy_score(y_test_uni, y_pred_rf_uni),2)
precision_rf_uni = round(precision_score(y_test_uni, y_pred_rf_uni, average='weighted'),2)
recall_rf_uni = round(recall_score(y_test_uni, y_pred_rf_uni, average='weighted'),2)
f1score_rf_uni = round(f1_score(y_test_uni, y_pred_rf_uni, average='weighted'),2)

akurasi_rf_bi = round(accuracy_score(y_test_bi, y_pred_rf_bi),2)
precision_rf_bi = round(precision_score(y_test_bi, y_pred_rf_bi, average='weighted'),2)
recall_rf_bi = round(recall_score(y_test_bi, y_pred_rf_bi, average='weighted'),2)
f1score_rf_bi = round(f1_score(y_test_bi, y_pred_rf_bi, average='weighted'),2)

akurasi_rf_tri = round(accuracy_score(y_test_tri, y_pred_rf_tri),2)
precision_rf_tri = round(precision_score(y_test_tri, y_pred_rf_tri, average='weighted'),2)
recall_rf_tri = round(recall_score(y_test_tri, y_pred_rf_tri, average='weighted'),2)
f1score_rf_tri = round(f1_score(y_test_tri, y_pred_rf_tri, average='weighted'),2)

## Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svm_classifier_uni = SVC(kernel = 'linear')
svm_classifier_bi = SVC(kernel = 'linear')
svm_classifier_tri = SVC(kernel = 'linear')
svm_classifier_uni.fit(X_train_uni, y_train_uni.ravel())
svm_classifier_bi.fit(X_train_bi, y_train_bi.ravel())
svm_classifier_tri.fit(X_train_tri, y_train_tri.ravel())

y_pred_svm_uni = svm_classifier_uni.predict(X_test_uni)
y_pred_svm_bi = svm_classifier_bi.predict(X_test_bi)
y_pred_svm_tri = svm_classifier_tri.predict(X_test_tri)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_svm_uni = round(accuracy_score(y_test_uni, y_pred_svm_uni),2)
precision_svm_uni = round(precision_score(y_test_uni, y_pred_svm_uni, average='weighted'),2)
recall_svm_uni = round(recall_score(y_test_uni, y_pred_svm_uni, average='weighted'),2)
f1score_svm_uni = round(f1_score(y_test_uni, y_pred_svm_uni, average='weighted'),2)

akurasi_svm_bi = round(accuracy_score(y_test_bi, y_pred_svm_bi),2)
precision_svm_bi = round(precision_score(y_test_bi, y_pred_svm_bi, average='weighted'),2)
recall_svm_bi = round(recall_score(y_test_bi, y_pred_svm_bi, average='weighted'),2)
f1score_svm_bi = round(f1_score(y_test_bi, y_pred_svm_bi, average='weighted'),2)

akurasi_svm_tri = round(accuracy_score(y_test_tri, y_pred_svm_tri),2)
precision_svm_tri = round(precision_score(y_test_tri, y_pred_svm_tri, average='weighted'),2)
recall_svm_tri = round(recall_score(y_test_tri, y_pred_svm_tri, average='weighted'),2)
f1score_svm_tri = round(f1_score(y_test_tri, y_pred_svm_tri, average='weighted'),2)

## LightGBM

In [ ]:
import lightgbm as lgb
lgb_classifier_uni = lgb.LGBMClassifier(random_state=42, boosting = 'gbdt', n_estimators=1000,
                                        tree_learner = 'voting', deterministic=True)
lgb_classifier_bi = lgb.LGBMClassifier(random_state=42, boosting = 'gbdt', n_estimators=1000,
                                       tree_learner = 'voting', deterministic=True)
lgb_classifier_tri = lgb.LGBMClassifier(random_state=42, boosting = 'gbdt', n_estimators=1000, 
                                        tree_learner = 'voting', deterministic=True)
lgb_classifier_uni.fit(X_train_uni, y_train_uni.ravel())
lgb_classifier_bi.fit(X_train_bi, y_train_bi.ravel())
lgb_classifier_tri.fit(X_train_tri, y_train_tri.ravel())

y_pred_lgb_uni = lgb_classifier_uni.predict(X_test_uni)
y_pred_lgb_bi = lgb_classifier_bi.predict(X_test_bi)
y_pred_lgb_tri = lgb_classifier_tri.predict(X_test_tri)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
akurasi_lgb_uni = round(accuracy_score(y_test_uni, y_pred_lgb_uni),2)
precision_lgb_uni = round(precision_score(y_test_uni, y_pred_lgb_uni, average='weighted'),2)
recall_lgb_uni = round(recall_score(y_test_uni, y_pred_lgb_uni, average='weighted'),2)
f1score_lgb_uni = round(f1_score(y_test_uni, y_pred_lgb_uni, average='weighted'),2)

akurasi_lgb_bi = round(accuracy_score(y_test_bi, y_pred_lgb_bi),2)
precision_lgb_bi = round(precision_score(y_test_bi, y_pred_lgb_bi, average='weighted'),2)
recall_lgb_bi = round(recall_score(y_test_bi, y_pred_lgb_bi, average='weighted'),2)
f1score_lgb_bi = round(f1_score(y_test_bi, y_pred_lgb_bi, average='weighted'),2)

akurasi_lgb_tri = round(accuracy_score(y_test_tri, y_pred_lgb_tri),2)
precision_lgb_tri = round(precision_score(y_test_tri, y_pred_lgb_tri, average='weighted'),2)
recall_lgb_tri = round(recall_score(y_test_tri, y_pred_lgb_tri, average='weighted'),2)
f1score_lgb_tri = round(f1_score(y_test_tri, y_pred_lgb_tri, average='weighted'),2)

# **Model Comparison**

In [ ]:
data_nb = pd.DataFrame({'Model':['Naive Bayes', 'Naive Bayes', 'Naive Bayes'],
                        'N-grams':['Unigrams', 'Bigrams', 'Trigrams'],
                        'Accuracy':[akurasi_nb_uni, akurasi_nb_bi, akurasi_nb_tri],
                        'Precision':[precision_nb_uni, precision_nb_bi, precision_nb_tri],
                        'Recall':[recall_nb_uni, recall_nb_bi, recall_nb_tri],
                        'F1-Score':[f1score_nb_uni, f1score_nb_bi, f1score_nb_tri]})

data_rf = pd.DataFrame({'Model':['Random Forest', 'Random Forest', 'Random Forest'],
                        'N-grams':['Unigrams', 'Bigrams', 'Trigrams'],
                        'Accuracy':[akurasi_rf_uni, akurasi_rf_bi, akurasi_rf_tri],
                        'Precision':[precision_rf_uni, precision_rf_bi, precision_rf_tri],
                        'Recall':[recall_rf_uni, recall_rf_bi, recall_rf_tri],
                        'F1-Score':[f1score_rf_uni, f1score_rf_bi, f1score_rf_tri]})

data_svm = pd.DataFrame({'Model':['Support Vector Machine', 'Support Vector Machine', 'Support Vector Machine'],
                        'N-grams':['Unigrams', 'Bigrams', 'Trigrams'],
                        'Accuracy':[akurasi_svm_uni, akurasi_svm_bi, akurasi_svm_tri],
                        'Precision':[precision_svm_uni, precision_svm_bi, precision_svm_tri],
                        'Recall':[recall_svm_uni, recall_svm_bi, recall_svm_tri],
                        'F1-Score':[f1score_svm_uni, f1score_svm_bi, f1score_svm_tri]})

data_lgbm = pd.DataFrame({'Model':['LightGBM', 'LightGBM', 'LightGBM'],
                        'N-grams':['Unigrams', 'Bigrams', 'Trigrams'],
                        'Accuracy':[akurasi_lgb_uni, akurasi_lgb_bi, akurasi_lgb_tri],
                        'Precision':[precision_lgb_uni, precision_lgb_bi, precision_lgb_tri],
                        'Recall':[recall_lgb_uni, recall_lgb_bi, recall_lgb_tri],
                        'F1-Score':[f1score_lgb_uni, f1score_lgb_bi, f1score_lgb_tri]})

data_performa = data_nb.append(data_rf).append(data_svm).append(data_lgbm)
data_performa

,Model,N-grams,Accuracy,Precision,Recall,F1-Score
0,Naive Bayes,Unigrams,0.83,0.84,0.83,0.83
1,Naive Bayes,Bigrams,0.72,0.79,0.72,0.71
2,Naive Bayes,Trigrams,0.56,0.73,0.56,0.46
0,Random Forest,Unigrams,0.91,0.91,0.91,0.91
1,Random Forest,Bigrams,0.77,0.77,0.77,0.77
2,Random Forest,Trigrams,0.56,0.74,0.56,0.47
0,Support Vector Machine,Unigrams,0.85,0.85,0.85,0.85
1,Support Vector Machine,Bigrams,0.75,0.79,0.75,0.74
2,Support Vector Machine,Trigrams,0.72,0.79,0.72,0.70
0,LightGBM,Unigrams,0.87,0.87,0.87,0.87


In [ ]:
data_performa.to_excel('Data_Klasifikasi_Hoax.xlsx')

# **Export Model**

In [ ]:
import pickle as pkl
file_model = 'Model_Final_Hoax.sav'
pkl.dump(rf_classifier_uni, open(file_model, 'wb'))

file_tfidf = 'TFIDF_Final_Hoax.pickle'
pkl.dump(cv_unigrams, open(file_tfidf, 'wb'))